In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\mayuo\\OneDrive\\Documents\\Machine Learning by Abraham\\MLOps wine'

##### Code the entity 

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    test_size: float
    bins: list
    labels: list
    target_column: str

##### Code the configuration manager

In [6]:
from src.MLOps_wine.constants import *
from src.MLOps_wine.utils.common import read_yaml, create_directories

In [20]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            test_size= config.test_size,
            bins=config.bins,
            labels=config.labels,
            target_column= schema.name
        )

        return data_transformation_config

##### We create the validation component

In [30]:
import os
from src.MLOps_wine import logger
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from joblib import dump

In [31]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        train,test = train_test_split(data,test_size=self.config.test_size)
        bins = self.config.bins  # Quality bins
        labels = self.config.labels  # Quality labels

        data[self.config.target_column] = pd.cut(data[self.config.target_column],bins=bins, labels=labels, include_lowest=True)

        numeric_features = list(train.select_dtypes(exclude="object").columns)
        numeric_features.remove('quality')
        numeric_transformer = StandardScaler()
        preprocessor = ColumnTransformer(
            [
                ("StandardScaler", numeric_transformer, numeric_features)
                
            ]
        )

        # Fit preprocessor on training data and transform both train and test data
        train_ = preprocessor.fit_transform(train.drop(self.config.target_column,axis=1))
        test_ = preprocessor.transform(test.drop(self.config.target_column,axis=1))

        # Convert transformed arrays back to DataFrame
        train = pd.DataFrame(train_, columns=numeric_features)
        test = pd.DataFrame(test_, columns=numeric_features)

        # Concatenate the target column to the DataFrame
        train[self.config.target_column] = data.loc[train.index, self.config.target_column]
        test[self.config.target_column] = data.loc[test.index, self.config.target_column]

        # Save preprocessed data to CSV files
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        dump(preprocessor, os.path.join(self.config.root_dir, "preprocessor.joblib"))

        logger.info(f"Data split into training and test sets (test_size: {self.config.test_size})")
        logger.info(f"Training features shape: {train.shape}")
        logger.info(f"Test features shape: {test.shape}")

        print(f"Data split into training and test sets (test_size: {self.config.test_size})")
        print(f"Training features shape: {train.shape}")
        print(f"Test features shape: {test.shape}")
    


In [32]:

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-04-29 18:09:20,087: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-29 18:09:20,090: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-29 18:09:20,095: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-29 18:09:20,097: INFO: common: created directory at: artifacts]
[2024-04-29 18:09:20,099: INFO: common: created directory at: artifacts/data_transformation]
[2024-04-29 18:09:20,192: INFO: 1721404121: Data split into training and test sets (test_size: 0.2)]
[2024-04-29 18:09:20,194: INFO: 1721404121: Training features shape: (1279, 12)]
[2024-04-29 18:09:20,196: INFO: 1721404121: Test features shape: (320, 12)]
Data split into training and test sets (test_size: 0.2)
Training features shape: (1279, 12)
Test features shape: (320, 12)
